In [1]:
import os
import sys
import matplotlib.pyplot as plt

os.chdir('..')

import numpy as np
import torch
import pandas as pd

from models.TrainPlanningOperator3D import PlanningOperator3D

from planners import generaterandompos
from planners import AStarPlanner, FMMPlanner, PlanningOperatorPlanner



Load Gibson Environment

In [2]:
Ntotal = 32
ntest = 8
sub = 1
Sx = 80
Sy = 80
Sz = 30

maps = np.load('dataset/final_occupancy_map.npy')
maps = maps[:Ntotal][-ntest:, ::sub, ::sub, ::sub][:, :Sx, :Sy, :Sz]



Generate Random Positions

In [3]:
startpositions = generaterandompos(maps)
goalpositions = generaterandompos(maps)

Test on Single Maps

In [4]:
mapx = maps[0,:,:,:]
startx = startpositions[0,:]
goalx = goalpositions[0,:]

# Model
modes = 6
width = 12
nlayers = 1

model = PlanningOperator3D(modes, modes, modes, width, nlayers)
model.load_state_dict(torch.load("/mountvol/dataset-80-15g/planningoperator_igibsonenv80_m6_w12_l1_b20_lr3e-3_15g_19sep/n480_lr3.000000e-03_gamma6.000000e-01_wd3.000000e-06_seed5/model3d.ckpt"))
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


success,pathcost,planningtime = AStarPlanner(startx, goalx, mapx)

print("\nAStar\nSuccess:", success,
      "\nPathCost:", pathcost,
      "\nPlanningTime:",planningtime)


success,pathcost,planningtime = FMMPlanner(startx, goalx, mapx)
print("\nFMM\nSuccess:", success,
      "\nPathCost:", pathcost,
      "\nPlanningTime:",planningtime)


success,pathcost,tot_time, sdf_time, inf_time, gd_time = PlanningOperatorPlanner(startx, goalx, mapx, model)
print("\nPlanning Operator\nSuccess:", success,
      "\nPathCost:", pathcost,
      "\nSDF Time:",sdf_time,
      "\nInference Time:",inf_time,
      "\nGradient Descent Time:",gd_time, 
      "\nTotal Time", tot_time)


/tmp/ipykernel_21000/1328886639.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/mountvol/dataset-80-15g/planningoperator_igibsonenv80


AStar
Success: True 
PathCost: 10.438793109825916 
PlanningTime: 0.007345438003540039

FMM
Success: True 
PathCost: 10.438793109825918 
PlanningTime: 0.04166769981384277

Planning Operator
Success: True 
PathCost: 10.438793109825918 
SDF Time: 0.04723072052001953 
Inference Time: 0.4409158229827881 
Gradient Descent Time: 0.0014371871948242188 
Total Time 0.48958444595336914
